### 수집된 포털 정보기반 딥크롤링
* 기존에 수집된 정보를 바탕으로 매체별 크롤링
* 다양한 서브 데이터의 정형화
* URL 기반의 다양한 크롤링 기법 사용

### 분석요소
* 제목 :
* 내용  :
* 작성일 :
* 매체: 
* 작성자:


### 기존 수집 데이터 활용


In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [2]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [3]:
calcDate('2021.2.1')

'2021-2-1 00:00:00.000000'

In [4]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [5]:
def getData(whr=''):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select nidx,ncomp,nurl from nnews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [6]:
def insSub(d):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from subnews where nkey="'+d[5]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into subnews (ncate,ntitle,ncomp,ndate,nurl,nkey) values(?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
        

In [7]:
def getAsia(urls):
    url = urls[2]
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('div.area_title')[0]
    wtitle=title_block.select('h3')[0].text.strip()
    writer=htmls.select('.article_view > .article .e_article')[0].text.strip()
    wday_=title_block.select('p.user_data')
    wday = str(wday_).split('</strong>')[-1].replace('</p>]','').replace('.','-').strip()
    article_=htmls.select('.article_view > .article ')[0]
    pattern='<!-- 본문// -->[\S\s]*'
    article__=re.sub(pattern,' ',str(article_))
#     pattern='<div id="bestnews_layer">.*'
#     article__=re.sub(pattern,' ',str(article__))
    warticle=bsp(article__,'html.parser').text.strip()
    return [wtitle,writer,wday,warticle,urls[0],urls[1],urls[2]]
    #return [wtitle,wday,urls[0],urls[1],urls[2]]
urls = (69,'아시아경제','https://view.asiae.co.kr/article/2021031715545233623')
#print(getAsia(urls))

In [8]:
def getDonga(urls):
    url = urls[2]
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('div.article_title')[0]
    wtitle=title_block.select('h1.title')[0].text.strip()
    writer=title_block.select('span.report')[0].text.replace('동아닷컴 기자','').strip()
    wday=title_block.select('span.date01')[-1].text.strip()
    article_=htmls.select('div.article_txt')[0]
    pattern='<script[^>]*>((\n|\r|.)*?)<\/script>'
    article__=re.sub(pattern,' ',str(article_))
    pattern='<div id="bestnews_layer">.*'
    article__=re.sub(pattern,' ',str(article__))
    warticle=bsp(article__,'html.parser').text
    return [wtitle,writer,wday,warticle,urls[0],urls[1],urls[2]]

In [9]:
def getDona(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('div.article_title')
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div.news_wrap')
        subartis=tgt.select('ul.list_cluster > li.sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a.news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a.news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a.sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a.sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)



In [10]:
allurl=getData()
allurl[:5]

[(1, '연합뉴스', 'http://yna.kr/AKR20210317071000030?did=1195m'),
 (2,
  '이데일리',
  'http://www.edaily.co.kr/news/newspath.asp?newsid=01430086628984304'),
 (3, '이투데이', 'https://www.etoday.co.kr/news/view/2005423'),
 (4, '파이낸셜뉴스', 'http://www.fnnews.com/news/202103180822274935'),
 (5, '뉴스1', 'https://www.news1.kr/articles/?4244392')]

In [11]:
for urls in allurl[:3]:
    print('-'*50)
    wdata=''
    #print(nidx,ncomp,turl)
    if urls[1]=="동아일보":
        wdata=getDonga(urls)
    elif urls[1]=="아시아경제":
        wdata=getAsia(urls)
    print(wdata)

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

